## Sequence-to-Sequence(seq2seq)

#### image-to-text, voice-to-text 등, 딥러닝을 통해 어떤 복잡한 것도 다른 것으로 바꿀 수 있다는 X-to-Y 모델에 대한 아이디어

다양한 RNN의 구성을 알아보기

인코더와 디코더 구조의 필요성 이해하기

교사 강요(teather forcing)의 원리 알기

훈련 단계와 추론 단계(inference)의 차이 알기

Rule based Machine Translation언어학 기반 기계 번역 - Statistical Machine Translation통계적 기계 번역 - Neural Machine Translation신경망 기계 번역

문장을 끝까지 인식한 후에 번역이나 통역을 해야 하기 때문에 입력 즉시 출력을 하는 것이 아니라 특정 time step에서 출력이 나오는 구조적 특징(many to many)

#### seq2seq_두 개의 RNN 아키텍처를 연결한 구조

입력문장을 받은 RNN인코더는 데이터 X를 해석하기 위한 저차원 feature vector z를 만들어내고, RNN디코더는 저차원의 feature z로부터 정보를 복원해서 다시 어떤 데이터 X'를 재생성.

feature vector z는 인코더RNN이 만들어낸 hidden state 벡터에 해당하고 디코더RNN에 전달하게 된다.(그렇기 때문에 인코더의 마지막 time step의 hidden state를 필요로 한다)

## Conditional Language Model

Text Generator문장 생성기는 Language Model언어모델을 구현한 것인데, 확률 모델의 특성상 어떤 단어가 생성될지 알 수 없다. 조건 C에 적합하도록 제어할 수 있다면 아주 유용한데, 

#### '프랑스어문장 Y를 생성하되, 해당 문장은 영어로 X라는 의미여야 해'에 해당. 문장X를 해석해서 C로 만드는 RNN인코더가 존재하는 형태.

## teacher forcing교사 강요

seq2seq는 훈련과정과 추론과정에서의 동작 방식이 다르다. 디코더RNN은 이전 time step의 출력을 현재 time step의 입력으로 사용한다.(추론시 동작방식)

훈련 과정에서 이전 time step이 잘못된 예측을 한다면, 이를 입력으로한 현재 time step의 예측도 잘못될 수 있고 이런 상황이 반복되면 훈련시간이 늘어나게된다. 훈련 과정에서는 정답 시퀀스를 알고 있기 때문에( 이전 time step의 예측값을 현재 time step의 입력으로 사용하지 않는다.)이전 time step의 실제값을 사용한다.(훈련데이터에 과대적합할 수 있음)

## 단어 수준의 번역과 문자 수준의 번역은 어느 쪽이 더 유리한가?

서로 간에 장단점이 있고, trade-off관계이기도 하다. 각 언어의 특성을 고려했을 때 전처리의 문제(비슷한 단어, 띄어쓰기...)를 쉽게 해결하려면 문자 수준으로 번역하면 되겠지만, 이 과정에서 단어 안에 내재된 정보가 소실될 수 있다. 문자가 단어를 이루는 패턴까지 학습해야 할 필요가 발생하기 때문에, 충분한 데이터가 확보되지 않았다면 문자 수준의 번역이 단어 수준의 번역보다 품질이 떨어진다.

#### 문자 수준의 번역기를 만들어보면, 단어 수준의 번역기는 쉽게 만들 수 있을 것이다.

## 1. 데이터 전처리

1-0 패키지 호출

1-1 해당파일을 데이터프레임으로 출력

1-2 불필요한 열을 제거, 훈련 데이터로 5000개만 이용하도록 설정

1-3 (seq2seq 동작을 위해) 디코더의 입력과 예측에 시작 토큰('\t')과 종료 토큰('\n')을 삽입

1-4 각 언어의 단어장 생성, (각 단어에 부여된 고유한 정수로 )텍스트 시퀀스를 정수 시퀀스로 인코딩

1-5 단어장 크기를 변수로 저장(0번 토큰을 고려하여 +1)

1-6 각 언어 데이터의 최대 길이를 확인

1-7 프랑스어 시퀀스의 경우, ①교사 강요를 위해 디코더의 입력으로 사용할 시퀀스(eos 토큰 필요없음), ②디코더의 출력과 비교할 정답 시퀀스(sos 토큰 필요없음)가 필요.

1-8 확인했던 각 언어 데이터의 최대 길이를 이용해서 패딩을 진행

1-9 각 정수에 대해 원-핫 인코딩 벡터화

1-10 데이터 50000건 3000건을 검증데이터로, 나머지를 학습데이터로 설정

In [1]:
# 패키지 호출

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
# 해당 파일을 데이터프레임으로 읽어온다

import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) # 샘플을 5개 출력

전체 샘플의 수 : 217975


,eng,fra,cc
183209,I have a very important decision to make.,Je dois prendre une décision très importante.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
133798,What did you do on the weekend?,Qu'as-tu fait le week-end ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
209713,Some people think the government has way too m...,Certaines personnes pensent que le gouvernemen...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
109675,We came so close to winning.,On est passés si près de la victoire.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
179431,I don't think that we can do that today.,Je ne pense pas que nous puissions le faire au...,CC-BY 2.0 (France) Attribution: tatoeba.org #7...


In [3]:
# 불필요한 3번째 열을 제거, 샘플의 수를 50000개로 설정

lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
47868,I'm friends with Tom.,Tom et moi sommes amis.
33476,I'm supposed to go.,Je suis censé partir.
10809,I am off today.,"Je suis en congé, aujourd'hui."
21612,Now I understand.,Maintenant je comprends.
17468,Tom scares easy.,Tom a la trouille facile.


In [4]:
# 디코더의 입력과 예측에 시작 토큰, 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
3204,I got fined.,\t On m'a collé une amende. \n
36079,We don't need that.,\t Nous n'avons pas besoin de ça. \n
42542,Tom collects comics.,\t Tom collectionne les comics. \n
4850,Good morning.,\t Bonjour. \n
13935,Are you smiling?,\t Êtes-vous en train de sourire ? \n


영어와 프랑스어 별도의 단어장을 만들고, 단어에 부여된 고유한 정수로 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 실행.

In [5]:
# 영어 단어장 만들기

eng_tokenizer = Tokenizer(char_level=True) # 문자 단위로 Tokenizer 생성
eng_tokenizer.fit_on_texts(lines.eng)       # eng 50000개의 행에 대해 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng) # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 4, 7], [19, 4, 7], [19, 4, 7]]

In [6]:
# 프랑스어 단어장 만들기

fra_tokenizer = Tokenizer(char_level=True) # 문자 단위로 토큰화
fra_tokenizer.fit_on_texts(lines.fra)       # fra 50000개의 행에 대해 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra) # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[10, 1, 19, 5, 1, 31, 1, 11],
 [10, 1, 15, 5, 12, 16, 29, 2, 14, 1, 11],
 [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]

In [7]:
# 단어장의 크기를 변수로 저장. 0번 토큰을 고려하여 +1

eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 52
프랑스어 단어장의 크기 : 73


In [8]:
# padding 을 위해 최대 길이를 확인
# 모델에 입력될 영어, 프랑스어 시퀀스의 길이가 일정해야 하기 때문에 최대길이를 맞추고 시퀀스 뒷부분은 패딩으로 채운다.

max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 21
프랑스어 시퀀스의 최대 길이 69


In [9]:
# 전체 통계 정보 출력

print('전체 샘플의 수 :', len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 52
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 21
프랑스어 시퀀스의 최대 길이 69


영어 시퀀스는 인코더의 입력으로 사용된다. 프랑스어 시퀀스의 경우는

#### 디코더의 출력과 비교할 정답 데이터(원래 목적)로 사용할 / Teacher forcing교사 강요를 위해 디코더의 입력으로 사용할

2가지 버전이 필요하다.

#### 디코더의 출력과 비교할 시퀀스는 sos토큰이 필요없고 / 디코더의 입력으로 사용할 시퀀스는 eos토큰이 필요없다.

In [10]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [11]:
# decoder의 입력과 출력을 각각 출력

print(decoder_input[:3])
print(decoder_target[:3])

# decoder의 입력은 11(eos토큰)이 제거되었고,
# decoder의 출력은 10(sos토큰)이 제거되었다.

[[10, 1, 19, 5, 1, 31, 1], [10, 1, 15, 5, 12, 16, 29, 2, 14, 1], [10, 1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 11], [1, 15, 5, 12, 16, 29, 2, 14, 1, 11], [1, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 1, 11]]


In [12]:
# padding
# decoder의 입력과 출력 시퀀스를 토큰 편집한 후, 모든 샘플들의 길이를 동일하게 변환, 시퀀스 뒷부분은 패딩으로 채운다.
# 영어 데이터의 모든 샘플들은 max_eng_seq_len의 길이, 프랑스어의 모든 샘플들은 max_fra_seq_len의 길이로 맞춘다.

encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 21)
프랑스어 입력데이터의 크기(shape) : (50000, 69)
프랑스어 출력데이터의 크기(shape) : (50000, 69)


In [13]:
# 인코더 샘플 출력

print(encoder_input[0])

[19  4  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [14]:
# 각 정수에 대해 원-핫 인코딩으로 벡터화

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

# 원-핫 인코딩은 각 정수를 단어장의 크기를 가지는 원-핫 벡터로 인코딩하기 때문에
# 원-핫 인코딩을 하고나서의 데이터의 크기는 (샘플의 수 x 샘플의 길이 x 단어장의 크기) 

영어 데이터의 크기(shape) : (50000, 21, 52)
프랑스어 입력데이터의 크기(shape) : (50000, 69, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 69, 73)


In [15]:
# 50000건 중 3000건은 검증데이터, 나머지는 학습데이터

n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 21, 52)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 69, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 69, 73)


## 2. Modeling_seq2seq

2-0 패키지 호출

2-1 encoder 설계(인코더 LSTM 셀의 마지막 time state의 hidden state와 cell state를, 디코더LSTM의 첫번째 hidden state와 cell state로 전달)

2-2 decoder 설계(인코더의 마지막 hidden state와 cell state를 저장해 두었다가 initial_state를 이용해서, LSTM셀의 초기 상태를 정의

2-3 decoder의 출력층을 설계(프랑스어 단어장의 크기를 기재, 활성화 함수로 소프트맥스를 사용)

2-4 compile(optimizer="rmsprop", loss="categorical_crossentropy")

2-5 학습

In [16]:
# 패키지 호출

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

일반적인 RNN의 경우에는, 인코더의 마지막 hidden state를 디코더의 첫번째 hidden state로 사용한다.

#### 인코더LSTM 셀의 마지막 time step의 hidden state와 cell state를 디코더LSTM의 첫번째 hidden state와 cell state로 전달해주어야 한다.

(전달하는 내용을 '리턴 시퀀스=트루, 리턴 스테이트=트루'로 표현한듯 하다)

In [17]:
# 입력 텐서 생성
encoder_inputs = Input(shape=(None, eng_vocab_size))

# hidden size가 256인 인코더 LSTM셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)

# 디코더로 전달할 hidden state, cell state를 리턴. encoder_output은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

In [18]:
# 디코더 설계

# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))

# hidden size가 256인 인코더의 LSTM셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)

# decoder_outputs은 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [19]:
# 디코더의 출력층 설계

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 316416      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [21]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
         validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
         batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 43s 19ms/step - loss: 0.9619 - val_loss: 0.8191
Epoch 2/50
368/368 [==============================] - 6s 16ms/step - loss: 0.5864 - val_loss: 0.6839
Epoch 3/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4919 - val_loss: 0.5936
Epoch 4/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4295 - val_loss: 0.5378
Epoch 5/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3899 - val_loss: 0.5092
Epoch 6/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3630 - val_loss: 0.4769
Epoch 7/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3407 - val_loss: 0.4554
Epoch 8/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3232 - val_loss: 0.4335
Epoch 9/50
368/368 [==============================] - 6s 17ms/step - loss: 0.3088 - val_loss: 0.4262
Epoch 10/50
368/368 [==============================] - 6s 17ms/step - loss: 0.2966 - val_l

## 3. Testing Model

3-1 encoder의 정의

3-2 decoder를 설계

3-3 decoder의 출력층을 재설계

3-4 단어에서 정수로, 정수에서 단어로 바꾸는 사전을 준비

3-5 예측 과정을 위한 함수를 구현(번역하고자 하는 문장의 정수 시퀀스를 입력하면, 내부의 인코더를 통해 마지막 시점의 hidden state를 리턴)

3-6 테스트

seq2seq는 훈련시와 테스트시 동작이 다르기 때문에, 테스트 단계의 디코더 모델은 설계를 다시 해줄 필요가 있다.

#### 테스트 단계에서의 디코더 동작 순서

#### 1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.

#### 2. sos 토큰인 '\t'를 디코더에 입력한다.

#### 3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.

#### 4. 3을 반복하다가 eos토큰인 '\n'이 예측되면 이를 중단한다.

이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 사용하는 단계를 추가해야 하기 때문에, 루프를 돌며 디코더의 LSTM셀을 수동으로 제어하는 느낌의 설계를 해야 한다.

In [22]:
# encoder를 정의(encoder_inputs와 encoder_states는 이미 정의한 것들을 재사용)

encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 52)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 316416    
Total params: 316,416
Trainable params: 316,416
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 디코더를 설계

# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))

# 이전 time step의 cell state를 저정하는 텐서
decoder_state_input_c = Input(shape=(256,))

# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states = [state_h, state_c]

In [24]:
# 디코더의 출력층을 재설계

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]              

In [25]:
# 단어에서 정수로, 정수에서 단어로 바꾸는 사전을 준비
# 문장을 숫자 인덱스로 바꾸는 Tokenizer를 만드는 과정에서 자동으로 만들어진 사전을 이미 갖고 있다.

eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

decode_sequence()의 입력으로 들어가는 것은 번역하고자 하는 문장의 정수 시퀀스.

decode sequence()내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 'input_seq'를 입력하면, encoder_model은 마지막 시점의 hidden state를 리턴한다.

hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 번역 문장을 완성하는 예측 과정을 진행.

디코더의 예측 과정은 이전 시점의 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행.

이 작업은 종료 토큰을 만나거나 주어진 최대 길이를 넘을 때까지 반복한다.

In [26]:
# 예측 과정을 위한 함수 decode_sequence()를 구현

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <sos>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ""
    
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이전 시점의 상태 state_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]
        
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        
        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        state_value = [h, c]
        
    return decoded_sentence

In [27]:
# 문장의 인덱스를 임의로 입력해서 출력 결과를 테스트

import numpy as np
for seq_index in [3,50,100,300,1001]: # 결과를 확인하고 싶은 문장의 인덱스(임의로 선정)
    input_seq = encoder_input[seq_index: seq_index +1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:                                                                      
-----------------------------------
입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:                                                                      
-----------------------------------
입력 문장: Got it!
정답 문장:  Compris ! 
번역기가 번역한 문장:                                                                      
-----------------------------------
입력 문장: Goodbye.
정답 문장:  Au revoir. 
번역기가 번역한 문장:                                                                      
-----------------------------------
입력 문장: Hands off.
정답 문장:  Pas touche ! 
번역기가 번역한 문장:                                                                      


# PROJECT_단어 Level로 번역기 업그레이드하기

In [28]:
# 주요 라이브러리의 버전 확인

import tensorflow

print(tensorflow.__version__)

2.6.0


동일한 데이터셋을 사용하되

#### ①글자 단위와는 다른 전처리,

#### ②to_categorical()함수가 아닌 Embedding layer임베딩 층을 추가하여

단어 단위의 번역기를 완성. 단어 단위로 번역기 진행시 글자 단위에 비해 단어장의 크기가 커지고, 학습 속도가 느려지기 때문에 원활한 진행을 위해 상위 33,000개의 샘플 중 3,000개는 테스트 데이터로 분리

### step1. 정제/정규화/전처리

#### Puntuation구두점의 분리

#### 소문자로 변환

#### 띄어쓰기 단위로 토큰화

### step2. 디코더의 문장에 시작 토큰과 종료 토큰 삽입



#### 7-7 샘플을 뽑아낼 때, '랜덤하게'뽑아내는 코드가 도대체 어디에 있는 것인가?;; sample( ) 함수는 'random'모듈에 속한 함수 중 하나로, 시퀀스에서 중복되지 않는 임의로 요소를 선택하여 반환하는 기능

#### EOS와 SOS를 없애는 시점과 이유에 대한 정리